In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import GoogleGenerativeAI
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict
import os

/home/rupeshchaudhari/Documents/LangGraph/LG/lg/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
load_dotenv()
api_key = os.getenv("api_key_gemini")

In [3]:
model = GoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=api_key)

In [12]:
# define state
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    score: int

In [5]:
def create_outline(state : BlogState) -> BlogState:
    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f"Create a detailed outline for a blog post titled: {title}"
    outline = model.invoke(prompt)

    # update state
    state['outline'] = outline

    return state

In [6]:
def create_blog(state : BlogState) -> BlogState:

    # fetch title and outline
    title = state['title']
    outline = state['outline']

    # call llm gen blog content
    prompt = f"Write a detailed blog on the title - {title} using following outline: \n {outline}"
    content = model.invoke(prompt)

    # update state
    state['content'] = content

    return state

In [13]:
def calculate_score(state : BlogState) -> BlogState:
    
    # fetch content
    outline = state['outline']
    content = state['content']

    # call llm to calculate the score
    prompt = f"based on the outline: {outline} and content: {content}, give a score out of 10 for the content, only a int value, eg. 5, 7, 10."
    score = model.invoke(prompt)

    # update state
    state['score'] = int(score)

    return state

In [14]:
# ceate graph
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('creat_blog', create_blog)
graph.add_node('calculate_score', calculate_score)


# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'creat_blog')
graph.add_edge('creat_blog', 'calculate_score')
graph.add_edge('calculate_score', END)

# compile
workflow = graph.compile()

In [15]:
intial_state = {'title':'Rise of AI in India'}

final_state = workflow.invoke(intial_state)

print(final_state) 
# if we do same thing using chains in langchain, we are not able to see outline

{'title': 'Rise of AI in India', 'outline': 'Here\'s a detailed outline for a blog post titled "Rise of AI in India," designed to be engaging, informative, and well-structured for a general audience interested in technology and global trends.\n\n---\n\n## Blog Post Outline: Rise of AI in India\n\n**Blog Post Title:** The Ascent of Algorithms: Unpacking the Rise of AI in India\n\n---\n\n### **I. Introduction: India\'s AI Ambition Takes Center Stage**\n\n*   **A. Catchy Hook:** Start with a surprising statistic, a bold statement about India\'s tech scene, or a relatable scenario where AI is already impacting daily life in India (e.g., UPI payments, smart city initiatives).\n    *   *Example:* "From bustling tech hubs to remote villages, the hum of artificial intelligence is growing louder across India, signaling a new era of innovation and transformation."\n*   **B. Defining AI (Briefly):** A quick, accessible explanation of what AI is for a general audience (not just robots, but intelli

In [16]:
print(final_state['score'])

10
